In [1]:
import kagglehub
path = kagglehub.dataset_download("garymk/movielens-25m-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/movielens-25m-dataset


In [3]:
import shutil
try:
  shutil.move("/kaggle/input/movielens-25m-dataset","/content/data")
except Exception as e:
  print(e)

Destination path '/content/data/movielens-25m-dataset' already exists


In [4]:
import pandas as pd

In [5]:
movies=pd.read_csv("/content/data/ml-25m/movies.csv")
movies['org_genres']=movies['genres']
movies['genres']=movies['genres'].str.split('|')
movies_exp=movies.explode('genres')
genres_onehot=pd.get_dummies(movies_exp['genres']).groupby(level=0).max()
movies=pd.concat([movies[['movieId','title','org_genres']],genres_onehot],axis=1)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(movies.iloc[:, 3:])

def recommend_movies(genre, top_n=10):
    genre_movies = movies[movies[genre] == 1]
    if genre_movies.empty:
        return []
    avg_similarity = similarity[genre_movies.index].mean(axis=0)
    ranked_indices = avg_similarity.argsort()[::-1]
    return movies.iloc[ranked_indices[:top_n]][['title', 'org_genres']]

In [15]:
movies.to_csv("movies_processed.csv",index=False)
import joblib
joblib.dump(similarity,"similarity.joblib")

['similarity.joblib']

In [7]:
import joblib
joblib.dump(similarity,"cos_similarity.joblib")

['cos_similarity.joblib']

In [17]:
input_genre = "Crime"
recommendations = recommend_movies(input_genre)
print(recommendations.head())

                                                   title            org_genres
40753                           To Catch a Killer (1992)  Crime|Drama|Thriller
109                                   Taxi Driver (1976)  Crime|Drama|Thriller
8904   Better Tomorrow II, A (Ying hung boon sik II) ...  Crime|Drama|Thriller
22196                                    Headshot (2011)  Crime|Drama|Thriller
14509  Varg Veum - Bitter Flowers (Varg Veum - Bitre ...  Crime|Drama|Thriller
